In [2]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms.ollama  import Ollama


llm = Ollama(
    model="mistral",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    base_url="http://192.168.86.250:11434",
)


llm.invoke("Tell me about the history of AI")

 The history of Artificial Intelligence (AI) can be traced back to the mid-20th century, although ideas and concepts related to machine intelligence date back much further. Here's a brief overview of some key milestones in the development of AI:

1. Alan Turing (1936-1950): Considered as the father of theoretical computer science and artificial intelligence, Turing proposed the concept of a "universal machine" that could perform any computable function. He also introduced the Turing test, which measures a machine's

KeyboardInterrupt: 

In [ ]:
%pip install unstructured["pdf"]

In [ ]:
from langchain.document_loaders import DirectoryLoader



loader = DirectoryLoader("test_loader",glob="*.pdf",show_progress=True)
docs = loader.load()

docs[0]

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader


loader = PyPDFDirectoryLoader("test_loader",glob="*.pdf")
data = loader.load()
data


In [ ]:
import os
os.environ['OPENAI_API_KEY']="sk-111111111111111111111111111111111111111111111111"
os.environ['OPENAI_API_BASE']="http://127.0.0.1:5001/v1"
import openai

response = openai.ChatCompletion.create(
  model="x",
  messages = [{ 'role': 'system', 'content': "Answer in a consistent style." },
    {'role': 'user', 'content': "Teach me about patience."},
    {'role': 'assistant', 'content': "The river that carves the deepest valley flows from a modest spring; the grandest symphony originates from a single note; the most intricate tapestry begins with a solitary thread."},
    {'role': 'user', 'content': "Teach me about the ocean."},
  ]
)
text = response['choices'][0]['message']['content']
print(text)

In [ ]:
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)
print(fibonacci(8)) # prints 8

In [ ]:

from langchain.document_loaders import PyPDFLoader





loader = PyPDFLoader("VerastreamModeltutorial.pdf",extract_images=True)
data = loader.load_and_split()

In [ ]:
!pip install pytube -q
!pip install youtube_transcript_api -q
!pip install amazon-textract-caller -q

In [ ]:
from langchain.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=-1sdWLr3TbI", add_video_info=True
)
loader.load()

In [ ]:
import boto3
from langchain.document_loaders import AmazonTextractPDFLoader


textract_client = boto3.client('textract')

file_path = "s3://s3-special-test-jy/VerastreamModeltutorial.pdf"
loader = AmazonTextractPDFLoader(file_path, client=textract_client)
documents = loader.load()

In [ ]:
from langchain.document_loaders import AmazonTextractImageLoader

In [ ]:

  
from langchain.embeddings import GPT4AllEmbeddings   
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma





loader = PyPDFLoader("VerastreamModeltutorial.pdf", extract_images=True)
data = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
all_splits = text_splitter.split_documents(data)
print(all_splits)




#vectorstore = Chroma.from_documents(persist_directory=documents=all_splits, embedding=GPT4AllEmbeddings())








In [ ]:
!pip install pandas -q

In [ ]:
import pandas as pd
import json


df = pd.json_normalize(json.loads(pd.DataFrame(data).to_json(orient="records")))

In [ ]:
df2 = pd.DataFrame(data)

In [ ]:
df2.head()

In [ ]:
%%HTML

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Team</th>
      <th>"Payroll (millions)"</th>
      <th>"Wins"</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Nationals</td>
      <td>81.34</td>
      <td>98</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Reds</td>
      <td>82.20</td>
      <td>97</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Yankees</td>
      <td>197.96</td>
      <td>95</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Giants</td>
      <td>117.62</td>
      <td>94</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Braves</td>
      <td>83.31</td>
      <td>94</td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:
import pprint
from langchain.chat_models import ChatOpenAI, ChatOllama
from langchain.chains import create_extraction_chain
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import BeautifulSoupTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import HumanMessage
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings.gpt4all import GPT4AllEmbeddings


class WebScraper:
    def __init__(self, model_name="gpt-3.5-turbo-0613", ollama_model=None):
        if ollama_model:
            self.llm = ChatOllama(
                model=ollama_model,
                callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
            )
        else:
            self.llm = ChatOpenAI(model=model_name)

    def extract(self, content: str, schema: dict):
        return create_extraction_chain(schema=schema, llm=self.llm).run(content)

    def scrape_with_playwright(self,urls, schema):
        loader = AsyncChromiumLoader(urls)
        docs = loader.load()
        bs_transformer = BeautifulSoupTransformer()
        docs_transformed = bs_transformer.transform_documents(
            docs, tags_to_extract=["span"]
        )
        print(f"Number of documents: {len(docs_transformed)}")
        print(docs_transformed[0].page_content)
        # Grab the first 1000 tokens of the site
        splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=1000, chunk_overlap=0
        )
        splits = splitter.split_documents(docs_transformed)
        vectorstore = Chroma.from_documents(persist_directory='testing_wsj',documents=splits, embedding=GPT4AllEmbeddings())

       
        # Process the first split
        extracted_content = self.extract(schema=schema, content=splits[0].page_content)
        pprint.pprint(extracted_content)
        return extracted_content

    def run_scraper(self, urls, schema):
        return self.scrape_with_playwright(urls, schema)

# Usage:
schema = {
    "properties": {
        "news_article_title": {"type": "string"},
        "news_article_summary": {"type": "string"},
    },
    "required": ["news_article_title", "news_article_summary"],
}

# Usage with ChatOpenAI:
#web_scraper_openai = WebScraper()
#extracted_content_openai = web_scraper_openai.run_scraper(["https://www.wsj.com"], schema)

# Usage with ChatOllama:
web_scraper_ollama = WebScraper(ollama_model="mistral:7b")
extracted_content_ollama = web_scraper_ollama.run_scraper(["https://www.wsj.com"], schema)
